In [ ]:
# 生成斜45度条纹图片
# 2025/03/18

from PIL import Image, ImageDraw

# 图片大小
width, height = 800, 800

# 创建空白图片
image = Image.new("RGB", (width, height), "white")
draw = ImageDraw.Draw(image)

# 定义四种颜色
colors = [
    (0,255,255),
    (255,0,0),
    (0,128,0)
]

# 条纹宽度
stripe_width = 50

# 绘制斜45度条纹
for x in range(width):
    for y in range(height):
        # 计算斜45度的位置
        pos = (x + y) // stripe_width
        # 根据位置选择颜色
        color = colors[pos % len(colors)]
        # 绘制像素
        draw.point((x, y), fill=color)

# 保存图片
image.save("diagonal_stripes.png")

# 显示图片
image.show()

In [ ]:
# 颜色百分比测试脚本
# 2025/03/18
from PIL import Image
from colorthief import ColorThief
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

def get_image_colors(image_path, num_colors=5):
    # 打开图像
    img = Image.open(image_path)
    
    # 转换为RGB模式
    img = img.convert('RGB')
    
    # 获取所有像素值
    pixels = np.array(img).reshape(-1, 3)
    
    # 计算每个颜色的出现次数
    pixel_counts = Counter(map(tuple, pixels))
    
    # 按出现次数排序
    sorted_pixel_counts = pixel_counts.most_common(num_colors)
    
    # 获取主颜色及其占比
    total_pixels = sum(count for color, count in sorted_pixel_counts)
    color_proportions = {f'#{r:02x}{g:02x}{b:02x}': count / total_pixels * 100 for (r, g, b), count in sorted_pixel_counts}
    
    return color_proportions


def show_color_distribution(image_path):
    # 获取颜色占比
    color_proportions = get_image_colors(image_path)
    
    # 显示颜色分布
    plt.figure(figsize=(6, 2))
    plt.bar(color_proportions.keys(), color_proportions.values(), color=color_proportions.keys())
    plt.title("Color Proportions")
    plt.xlabel("Color")
    plt.ylabel("Percentage")
    plt.show()

# 你可以将此处的路径更改为你自己的图像路径
image_path = "./flag/turkey.png"

# 获取图像的颜色占比
color_proportions = get_image_colors(image_path, num_colors=5)

# 输出颜色占比
print(color_proportions)

# 显示颜色占比图表
show_color_distribution(image_path)


In [ ]:

# 打印图例
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
from matplotlib.lines import Line2D

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False 

world = gpd.read_file('./worldmap4/world.json')
world = world.to_crs(ccrs.PlateCarree())
fig, ax = plt.subplots(1, 1, figsize=(15, 10), subplot_kw={'projection': ccrs.PlateCarree()})

legend_elements = [
    Line2D([0], [0], marker='s', color='k', markerfacecolor='lightblue', markersize=10, label='好莱坞环球影城', linestyle='None'),
    Line2D([0], [0], marker='s', color='k', markerfacecolor='lightgreen', markersize=10, label='奥兰多环球影城度假村', linestyle='None'),
    Line2D([0], [0], marker='s', color='k', markerfacecolor='lightcoral', markersize=10, label='日本环球影城', linestyle='None'),
    Line2D([0], [0], marker='s', color='k', markerfacecolor='lightyellow', markersize=10, label='北京环球度假区', linestyle='None'),
    Line2D([0], [0], marker='s', color='k', markerfacecolor='lightpink', markersize=10, label='新加坡环球影城', linestyle='None'),
    Line2D([0], [0], marker='o', color='k', markerfacecolor='lightgrey', markersize=10, label='英国环球影城(在建)', linestyle='None')
]
legend = ax.legend(handles=legend_elements, loc='center', title='图例', title_fontsize=16,prop={'size': 14})
legend.get_frame().set_facecolor('lightgray')

# 设置横纵坐标刻度不可见
ax.tick_params(axis='both', which='both', length=0, labelsize=0)
ax.set_axis_off()

# 保存图像并显示
plt.savefig('test.png', dpi=1200, bbox_inches='tight', pad_inches=0, transparent=True)
plt.show()


In [8]:
import requests
import pandas as pd

def get_gaode_parks(api_key, city=None, output_file='people_park.csv'):
    base_url = "https://restapi.amap.com/v3/place/text"
    
    # 如果未指定城市，则搜索全国
    # if city is None:
    #     city = "中国"
    
    params = {
        'key': api_key,
        'keywords': '人民公园',
        'types': '110101',
        # 'city': city,
        'offset': 25,  # 每页记录数
        'page': 1,      # 页码
        'extensions': 'base'
    }
    
    all_parks = []
    
    while True:
        response = requests.get(base_url, params=params)
        data = response.json()
        print(data)  # 打印返回的JSON数据
        if data['status'] == '1' and int(data['count']) > 0:
            for poi in data['pois']:
                park_info = {
                    'name': poi['name'],
                    'address': poi['address'],
                    'province': poi['pname'],
                    'city': poi['cityname'],
                    'district': poi['adname'],
                    'longitude': poi['location'].split(',')[0],
                    'latitude': poi['location'].split(',')[1]
                }
                all_parks.append(park_info)
            
            if len(data['pois']) < params['offset']:
                break
            params['page'] += 1
        else:
            break
    
    # 保存到CSV文件
    df = pd.DataFrame(all_parks)
    df.to_csv(output_file, index=False, encoding='utf_8_sig')
    return df

# 使用示例
api_key = "fe09ffacf890bf1c9471054ce5dc122d"
# 可以指定城市或搜索全国
parks_df = get_gaode_parks(api_key, city="全国")
print(parks_df.head())

{'suggestion': {'keywords': [], 'cities': []}, 'count': '5', 'infocode': '10000', 'pois': [{'parent': [], 'address': '金彭西路375号', 'distance': [], 'pname': '四川省', 'importance': [], 'biz_ext': [], 'biz_type': 'tour', 'cityname': '成都市', 'type': '风景名胜;公园广场;公园', 'photos': [], 'typecode': '110101', 'shopinfo': '2', 'poiweight': [], 'childtype': [], 'adname': '彭州市', 'name': '彭州园', 'location': '103.931338,30.986038', 'tel': '028-83871232', 'shopid': [], 'id': 'B001C02539'}, {'parent': [], 'address': '望江路30号', 'distance': [], 'pname': '四川省', 'importance': [], 'biz_ext': [], 'biz_type': 'tour', 'cityname': '成都市', 'type': '风景名胜;公园广场;公园', 'photos': [], 'typecode': '110101', 'shopinfo': '2', 'poiweight': [], 'childtype': [], 'adname': '武侯区', 'name': '望江楼公园', 'location': '104.092217,30.629277', 'tel': '028-85223389', 'shopid': [], 'id': 'B001C06R0O'}, {'parent': [], 'address': '蜀源大道1999号', 'distance': [], 'pname': '四川省', 'importance': [], 'biz_ext': [], 'biz_type': 'tour', 'cityname': '成都市', 'type': 